In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,to_date,month,avg,min,stddev,variance

In [2]:
spark = SparkSession.builder.appName("Process").getOrCreate()
spark

In [4]:
data = spark.read.csv("F:/test3.csv",header=True,inferSchema=True)


In [6]:
data.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [7]:
data.describe().show()

+-------+-----+-----------+-----------------+
|summary| Name|Departments|           salary|
+-------+-----+-----------+-----------------+
|  count|   10|         10|               10|
|   mean| NULL|       NULL|           7300.0|
| stddev| NULL|       NULL|5396.500923952689|
|    min|Krish|   Big Data|             2000|
|    max|Sunny|        IOT|            20000|
+-------+-----+-----------+-----------------+



In [8]:
data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



In [22]:
data.groupBy('Departments').agg

<bound method GroupedData.agg of GroupedData[grouping expressions: [Departments], value: [Name: string, Departments: string ... 1 more field], type: GroupBy]>

In [38]:
fc_transaction_base = spark.read.csv("C:/Users/acer/Downloads/fc_transaction_base.csv",header=True,inferSchema=True)

In [39]:
fc_transaction_base.show(5)

+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+
| tran_date|     account_number|branch|product|lcy_amount|transaction_code|        description1|dc_indicator|is_salary|
+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+
|09/06/2020|02XYZXYZ10017529992|    15|    SBA|   14500.0|              CI|ATM WDL /00463701...|    withdraw|        0|
|14/06/2020|02XYZXYZ10017529992|    15|    SBA|   20000.0|              CI|ATM WDL /00463701...|    withdraw|        0|
|01/06/2020|02XYZXYZ10017517823|    15|    SBA|    4000.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|
|05/06/2020|02XYZXYZ10017517823|    15|    SBA|     500.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|
|12/06/2020|02XYZXYZ10017517823|    15|    SBA|    5000.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|
+----------+-------------------+------+-

In [40]:
fc_transaction_base.printSchema()

root
 |-- tran_date: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- branch: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- lcy_amount: double (nullable = true)
 |-- transaction_code: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- dc_indicator: string (nullable = true)
 |-- is_salary: integer (nullable = true)



In [41]:
fc_account_master = spark.read.csv("C:/Users/acer/Downloads/assignment_2/assignment_2/fc_account_master.csv",header=True,inferSchema=True)

In [42]:
fc_account_master.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- customer_code: string (nullable = true)
 |-- product: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- acc_open_date: date (nullable = true)
 |-- acc_closed_date: string (nullable = true)
 |-- active_flag: integer (nullable = true)



In [43]:
fc_transaction_base = fc_transaction_base.withColumn("tran_date", to_date(col("tran_date"), "dd/MM/yyyy"))

In [44]:
fc_transaction_base.printSchema()

root
 |-- tran_date: date (nullable = true)
 |-- account_number: string (nullable = true)
 |-- branch: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- lcy_amount: double (nullable = true)
 |-- transaction_code: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- dc_indicator: string (nullable = true)
 |-- is_salary: integer (nullable = true)



In [45]:
fc_transaction_base = fc_transaction_base.withColumn("month",month(col("tran_date")))

In [46]:
fc_transaction_base.show(5)

+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+-----+
| tran_date|     account_number|branch|product|lcy_amount|transaction_code|        description1|dc_indicator|is_salary|month|
+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+-----+
|2020-06-09|02XYZXYZ10017529992|    15|    SBA|   14500.0|              CI|ATM WDL /00463701...|    withdraw|        0|    6|
|2020-06-14|02XYZXYZ10017529992|    15|    SBA|   20000.0|              CI|ATM WDL /00463701...|    withdraw|        0|    6|
|2020-06-01|02XYZXYZ10017517823|    15|    SBA|    4000.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|    6|
|2020-06-05|02XYZXYZ10017517823|    15|    SBA|     500.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|    6|
|2020-06-12|02XYZXYZ10017517823|    15|    SBA|    5000.0|              CI|ATM WDL /NICAV701...|    withdraw|        0

In [64]:
fc_transaction_base.select("dc_indicator").distinct().show()

+------------+
|dc_indicator|
+------------+
|     deposit|
|    withdraw|
|        NULL|
+------------+



In [65]:
fc_transaction_base = fc_transaction_base.na.drop(subset="dc_indicator")

In [66]:
fc_transaction_base.select("dc_indicator").distinct().show()

+------------+
|dc_indicator|
+------------+
|     deposit|
|    withdraw|
+------------+



In [67]:
fc_trans_avg_salary = fc_transaction_base.groupBy('month','dc_indicator').agg(avg("lcy_amount").alias("AVerage_Monthly"),stddev("lcy_amount").alias("std_Monthly"),variance("lcy_amount").alias("variance_Monthly"))

In [68]:
fc_trans_avg_salary.show()

+-----+------------+------------------+------------------+--------------------+
|month|dc_indicator|   AVerage_Monthly|       std_Monthly|    variance_Monthly|
+-----+------------+------------------+------------------+--------------------+
|    7|    withdraw|29542.774370950818|206448.55792322033|4.262100706857725...|
|    8|     deposit| 56295.85522375221|273560.86691461364|7.483554790707497E10|
|    6|     deposit|  99075.4756339712| 319613.3873055106|1.021527173449023E11|
|    7|     deposit| 72276.67343830765|334540.14597747347|1.119171092706292...|
|    8|    withdraw|25084.110745400227|171971.27718957173|2.957412017821251...|
|    6|    withdraw|23076.009773670306|141984.58293541646|2.015962179134416E10|
+-----+------------+------------------+------------------+--------------------+



In [69]:
fc_trans_avg_salary = fc_transaction_base.groupBy('month').pivot('dc_indicator').agg(avg("lcy_amount").alias("AVerage_Monthly"),stddev("lcy_amount").alias("std_Monthly"),variance("lcy_amount").alias("variance_Monthly"))

In [70]:
fc_trans_avg_salary.show()

+-----+-----------------------+-------------------+------------------------+------------------------+--------------------+-------------------------+
|month|deposit_AVerage_Monthly|deposit_std_Monthly|deposit_variance_Monthly|withdraw_AVerage_Monthly|withdraw_std_Monthly|withdraw_variance_Monthly|
+-----+-----------------------+-------------------+------------------------+------------------------+--------------------+-------------------------+
|    6|       99075.4756339712|  319613.3873055106|    1.021527173449023E11|      23076.009773670306|  141984.58293541646|     2.015962179134416E10|
|    8|      56295.85522375221| 273560.86691461364|    7.483554790707497E10|      25084.110745400227|  171971.27718957173|     2.957412017821251...|
|    7|      72276.67343830765| 334540.14597747347|    1.119171092706292...|      29542.774370950818|  206448.55792322033|     4.262100706857725...|
+-----+-----------------------+-------------------+------------------------+------------------------+-----

In [88]:
joined = fc_transaction_base.join(fc_account_master,fc_transaction_base["account_number"]==fc_account_master["account_number"],"inner").select(fc_transaction_base["account_number"],fc_transaction_base["lcy_amount"],fc_transaction_base["month"],fc_transaction_base["dc_indicator"],fc_account_master["customer_code"])

In [89]:
joined.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- lcy_amount: double (nullable = true)
 |-- month: integer (nullable = true)
 |-- dc_indicator: string (nullable = true)
 |-- customer_code: string (nullable = true)



In [90]:
final_result = joined.groupBy('account_number','month').pivot('dc_indicator').agg(avg("lcy_amount").alias("AVerage_Monthly"),stddev("lcy_amount").alias("std_Monthly"),variance("lcy_amount").alias("variance_Monthly"))

In [92]:
final_result.show(5)

+-------------------+-----+-----------------------+-------------------+------------------------+------------------------+--------------------+-------------------------+
|     account_number|month|deposit_AVerage_Monthly|deposit_std_Monthly|deposit_variance_Monthly|withdraw_AVerage_Monthly|withdraw_std_Monthly|withdraw_variance_Monthly|
+-------------------+-----+-----------------------+-------------------+------------------------+------------------------+--------------------+-------------------------+
|02XYZXYZ10017514877|    6|               111080.0|               NULL|                    NULL|                21039.25|  19743.481990900185|          3.89805081125E8|
|02XYZXYZ10017542037|    7|                  11.16|               NULL|                    NULL|                    0.56|                NULL|                     NULL|
|02XYZXYZ10017516724|    7|                1530.74|               NULL|                    NULL|      208.84666666666666|   117.2765557702533|       13753.